# Graph Neural Networks for Decentralized Multi-Robot Path Planning In Warehouse

### Importing the libraries

In [1]:
from Dataset_Generator import DatasetGenerator 
import torch
import torch
import numpy as np
import random

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # for multi-GPU setups

# Optional: Force deterministic behavior on GPU (may impact performance)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Part 1 - Dataset Generation

In [2]:
grid = [[0 for _ in range(20)] for _ in range(20)]  # Generate grid map
dataset_generator = DatasetGenerator(num_cases=5000, num_agents=6, grid=grid)
# cases = dataset_generator.generate_cases()
# dataset_generator.save_cases_to_file(cases, "dataset.json")
# print(f"Generated and saved {len(cases)} cases.")
cases = dataset_generator.load_cases_from_file("dataset.json")

### Part 2 - Pre-Processing

In [3]:
from pre_processing import Preprocessing
p = Preprocessing(grid,cases,3)
data_tensors  = p.begin()

In [4]:
# with open("data_tensors.txt", 'w') as file:
#     file.write(str(data_tensors))

### Part 3 - Encoding

In [5]:
from Encoder import Encode
encoder = Encode(data_tensors,6)
encoded_tensors = encoder.begin()

In [6]:
# with open("encoded_tensors.txt", 'w') as file:
#     file.write(str(encoded_tensors))

### Part 4 - GNN

##### Get Adjacency Matrix

In [7]:
from Adjacency_Matrix import adj_mat
adj = adj_mat(cases,3)
adj_matrices = adj.get_adj_mat()

In [8]:
# with open("Adj_Matrices.txt", 'w') as file:
#     file.write(str(adj_matrices))

##### Create GNN Model

In [9]:
from GNN_file import Communication_GNN
comm_gnn = Communication_GNN(encoded=encoded_tensors, adj_mat=adj_matrices, num_of_agents=6)
gnn_features = comm_gnn.begin()

c:\Users\moham\anaconda3\Lib\site-packages\torch_geometric\typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "


In [10]:
# with open("GNN_Features.txt", 'w') as file:
#     file.write(str(gnn_features))

##### Extract Actions

In [11]:
from Extarct_Actions import Action_Extractor
action_extractor = Action_Extractor(cases, 6)
actions = action_extractor.extract()

In [12]:
gnn_features = torch.cat([gnn_features[key][subkey] for key in gnn_features for subkey in gnn_features[key]])

In [13]:
actions = torch.tensor([actions[key][subkey] for key in actions for subkey in actions[key]])

In [14]:
reshaped_gnn_features = []
temp = []
c = 1
for i in range(len(gnn_features)):
    temp.append(gnn_features[i].tolist())
    if c % 6 == 0:
        reshaped_gnn_features.append(torch.Tensor(temp))
        temp = []
    c+=1

In [15]:
dataset = []
for i in range(len(actions)):
    dataset.append((reshaped_gnn_features[i],actions[i]))

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ActionMLP(nn.Module):
    def __init__(self, input_dim=128, hidden_dim=128, num_actions=5):
        super(ActionMLP, self).__init__()
        seed = 42
        torch.manual_seed(seed)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        seed = 42
        torch.manual_seed(seed)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        seed = 42
        torch.manual_seed(seed)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        seed = 42
        torch.manual_seed(seed)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        seed = 42
        torch.manual_seed(seed)
        self.fc3 = nn.Linear(hidden_dim, num_actions)
        seed = 42
        torch.manual_seed(seed)
        self.dropout = nn.Dropout(0.2)  # Dropout for regularization

    def forward(self, x):
        seed = 42
        torch.manual_seed(seed)
        x = F.relu(self.bn1(self.fc1(x)))
        seed = 42
        torch.manual_seed(seed)
        x = self.dropout(x)
        seed = 42
        torch.manual_seed(seed)
        x = F.relu(self.bn2(self.fc2(x)))
        seed = 42
        torch.manual_seed(seed)
        x = self.dropout(x)
        seed = 42
        torch.manual_seed(seed)
        x = self.fc3(x)
        seed = 42
        torch.manual_seed(seed)
        return F.softmax(x, dim=-1)

In [17]:
# from MLP_Action import ActionMLP
mlp = ActionMLP(input_dim= 128, hidden_dim= 128, num_actions= 5)

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
class Generate_Model:
    def __init__(self,model, dataset, num_epochs = 150):
        self.model = model
        self.num_epochs = num_epochs
        seed = 42
        torch.manual_seed(seed)
        self.criterion = nn.CrossEntropyLoss()
        torch.manual_seed(seed)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        seed = 42
        torch.manual_seed(seed)
        self.model.to(self.device)
        train_size = int(0.7 * len(dataset))
        val_size = int(0.15 * len(dataset))
        test_size = len(dataset) - train_size - val_size
        train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
        self.train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        self.val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
        self.test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        
    def evaluate_model(self, model, dataloader, device, criterion):
        seed = 42
        torch.manual_seed(seed)
        model.eval()  # Set model to evaluation mode
        total_loss = 0.0
        total_correct = 0
        total_samples = 0
        with torch.no_grad():
            for gnn_feature, target_actions in dataloader:
                # observations: [batch_size, num_agents, channels, height, width]
                batch_size, num_agents = gnn_feature.shape[0], gnn_feature.shape[1]
                # Flatten observations: [batch_size*num_agents, channels, height, width]
                gnn_feature = gnn_feature.view(batch_size * num_agents,
                                             gnn_feature.shape[2]).to(device)
                # Flatten target actions: [batch_size*num_agents]
                target_actions = target_actions.view(-1).to(device)
                # Move edge_index to device
                torch.manual_seed(seed)
                # Forward pass
                predictions = model(gnn_feature)  # [batch_size*num_agents, num_actions]
                torch.manual_seed(seed)
                loss = criterion(predictions, target_actions)
                torch.manual_seed(seed)
                total_loss += loss.item() * target_actions.size(0)
                torch.manual_seed(seed)
                # Compute accuracy
                predicted_labels = torch.argmax(predictions, dim=1)
                total_correct += (predicted_labels == target_actions).sum().item()
                total_samples += target_actions.size(0)
                
        avg_loss = total_loss / total_samples
        accuracy = total_correct / total_samples
        return avg_loss, accuracy
    
    def train_model(self):
        seed = 42
        torch.manual_seed(seed)
        optimizer = optim.AdamW(self.model.parameters(), lr=1e-3, betas=(0.9, 0.999), weight_decay=1e-5)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)
        for epoch in range(1, self.num_epochs + 1):
            self.model.train()
            running_loss = 0.0
            for gnn_feature, target_action in self.train_loader:
                seed = 42
                torch.manual_seed(seed)
                batch_size, num_agents = gnn_feature.shape[0], gnn_feature.shape[1]
                gnn_feature = gnn_feature.view(batch_size * num_agents,
                                                gnn_feature.shape[2]).to(self.device)
                target_action = target_action.view(-1).to(self.device)
                torch.manual_seed(seed)
                optimizer.zero_grad()
                torch.manual_seed(seed)
                predictions = self.model(gnn_feature)
                torch.manual_seed(seed)
                loss = self.criterion(predictions, target_action)
                torch.manual_seed(seed)
                loss.backward()
                torch.manual_seed(seed)
                optimizer.step()
                torch.manual_seed(seed)
                running_loss += loss.item()
                torch.manual_seed(seed)
            avg_train_loss = running_loss / len(self.train_loader)
            val_loss, val_accuracy = self.evaluate_model(self.model, self.val_loader, self.device, self.criterion)
            scheduler.step(val_loss)
            print(f"Epoch [{epoch}/{self.num_epochs}] - "
                  f"Train Loss: {avg_train_loss:.4f} - "
                  f"LR: {scheduler.get_last_lr()[0]:.6f} - "
                  f"Val Loss: {val_loss:.4f} - "
                  f"Val Acc: {val_accuracy * 100:.2f}%")
        self.save_model()
    def test_model(self):
        seed = 42
        torch.manual_seed(seed)
        test_loss, test_accuracy = self.evaluate_model(self.model, self.test_loader, self.device, self.criterion)
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%")
        return test_loss, test_accuracy
    def save_model(self):
        torch.save(self.model.state_dict(), "trained_model_3.pth")
        print("Model saved successfully!")

In [19]:
# from Model_Generation import Generate_Model
model = Generate_Model(model = mlp, dataset=dataset,num_epochs=200)

In [20]:
model.train_model()

c:\Users\moham\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/200] - Train Loss: 1.1091 - LR: 0.001000 - Val Loss: 1.0958 - Val Acc: 80.46%
Epoch [2/200] - Train Loss: 1.0909 - LR: 0.001000 - Val Loss: 1.0903 - Val Acc: 80.98%
Epoch [3/200] - Train Loss: 1.0863 - LR: 0.001000 - Val Loss: 1.0857 - Val Acc: 81.48%
Epoch [4/200] - Train Loss: 1.0832 - LR: 0.001000 - Val Loss: 1.0851 - Val Acc: 81.57%
Epoch [5/200] - Train Loss: 1.0807 - LR: 0.001000 - Val Loss: 1.0829 - Val Acc: 81.77%
Epoch [6/200] - Train Loss: 1.0783 - LR: 0.001000 - Val Loss: 1.0826 - Val Acc: 81.80%
Epoch [7/200] - Train Loss: 1.0763 - LR: 0.001000 - Val Loss: 1.0824 - Val Acc: 81.85%
Epoch [8/200] - Train Loss: 1.0746 - LR: 0.001000 - Val Loss: 1.0835 - Val Acc: 81.76%
Epoch [9/200] - Train Loss: 1.0730 - LR: 0.001000 - Val Loss: 1.0821 - Val Acc: 81.91%
Epoch [10/200] - Train Loss: 1.0715 - LR: 0.001000 - Val Loss: 1.0829 - Val Acc: 81.84%
Epoch [11/200] - Train Loss: 1.0700 - LR: 0.001000 - Val Loss: 1.0814 - Val Acc: 81.99%
Epoch [12/200] - Train Loss: 1.0688 - LR:

In [22]:
model.test_model()

Test Loss: 1.0801, Test Accuracy: 82.28%


(1.08011870552934, 0.8228367649037814)

In [ ]:
# new_model = ActionMLP(input_dim= 128, hidden_dim= 128, num_actions= 5)
# state_dict = torch.load('trained_model_3.pth')
# new_model.load_state_dict(state_dict)
# new_model.eval()  # set the model to evaluation mode

In [146]:
new_dataset_generator = DatasetGenerator(num_cases=100, num_agents=6, grid=grid)
eval_cases = new_dataset_generator.generate_cases()

case  0
case added
case  1
case added
case  2
case added
case  3
case added
case  4
case added
case  5
case added
case  6
case added
case  7
case added
case  8
case added
case  9
case added
case  10
case added
case  11
case added
case  12
case added
case  13
case added
case  14
case added
case  15
case added
case  16
case added
case  17
case added
case  18
case added
case  19
case added
case  20
case added
case  21
case added
case  22
case added
case  23
case added
case  24
case added
case  25
case added
case  26
case added
case  27
case added
case  28
case added
case  29
case added
case  30
case added
case  31
case added
case  32
case added
case  33
case added
case  34
case added
case  35
case added
case  36
case added
case  37
case added
case  38
case added
case  39
case added
case  40
case added
case  41
case added
case  42
case added
case  43
case added
case  44
case added
case  45
case added
case  46
case added
case  47
case added
case  48
case added
case  49
case added
case  50
c

In [ ]:
print(eval_cases[0]['start_positions'])
print(eval_cases[0]['goal_positions'])
print(eval_cases[0]['paths'])

[(17, 9), (0, 16), (11, 16), (16, 14), (15, 1), (9, 6)]
[(10, 16), (2, 3), (5, 17), (17, 0), (2, 6), (6, 13)]
[[(17, 9), (16, 9), (16, 10), (15, 10), (15, 11), (14, 11), (14, 12), (13, 12), (13, 13), (12, 13), (12, 14), (11, 14), (11, 15), (10, 15), (10, 16)], [(0, 16), (0, 15), (0, 14), (0, 13), (0, 12), (0, 11), (0, 10), (0, 9), (0, 8), (0, 7), (0, 6), (0, 5), (0, 4), (1, 4), (1, 3), (2, 3)], [(11, 16), (10, 16), (9, 16), (8, 16), (7, 16), (6, 16), (5, 16), (5, 17)], [(16, 14), (16, 13), (16, 12), (16, 11), (16, 10), (16, 9), (16, 8), (16, 7), (16, 6), (16, 5), (16, 4), (16, 3), (16, 2), (16, 1), (16, 0), (17, 0)], [(15, 1), (14, 1), (13, 1), (12, 1), (11, 1), (10, 1), (9, 1), (8, 1), (7, 1), (6, 1), (6, 2), (5, 2), (5, 3), (4, 3), (4, 4), (3, 4), (3, 5), (2, 5), (2, 6)], [(9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (8, 10), (8, 11), (7, 11), (7, 12), (6, 12), (6, 13)]]


In [148]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_of_robots = 1
success = 0
actions = [[0,0],[-1,0],[0,1],[1,0],[0,-1]]
# actions = [[0,0],[-1,0],[0,-1],[1,0],[0,1]]
seed = 42
torch.manual_seed(seed)
with torch.inference_mode():
    for case in eval_cases:
        threshold = 0
        for path in case['paths']:
            threshold+= len(path)
        threshold = 3*threshold # from the paper
        num_of_steps = 0
        robots_reached = set()
        step_number = 0
        current_nodes = []
        path_robot_zero = []
        while num_of_steps <= threshold and len(robots_reached) <=6:
            paths = []
            goals = []
            for robot in range(num_of_robots):
                if step_number == 0:
                    current_node = case['paths'][robot][step_number]
                    current_nodes.append(list(current_node))
                    if robot == 0:
                        print(current_node)
                        print(list(case['paths'][robot][-1]))
                    paths.append([list(current_node)])
                    goals.append(list(case['goal_positions'][robot]))
                else:
                    paths.append([current_nodes[robot]])
                    goals.append(list(case['goal_positions'][robot]))
            # print(current_nodes[0])
            step_case = {"start_positions": current_nodes, "goal_positions": goals,"paths":paths}
            # print(paths)
            # print("start_preprocessing")
            
            p_new = Preprocessing(grid,[step_case],3)
            data_tensors_new  = p_new.begin()
            # if num_of_steps == 11:
            #     # print(len(step_case['start_positions']))
            #     print(step_case)
            #     print(current_nodes[0])
            #     print(data_tensors_new[0]['channel 2'])
                
            # print("start_encoding")
            # torch.manual_seed(42)
            new_encoder = Encode(data_tensors_new,num_of_robots)
            new_encoded_tensors = new_encoder.begin()
            # print(new_encoded_tensors[0])
            # print("start adj")
            new_adj = adj_mat([step_case],3)
            new_adj_matrices = new_adj.get_adj_mat()
            # print("start gnn")
            new_comm_gnn = Communication_GNN(encoded=new_encoded_tensors, adj_mat=new_adj_matrices, num_of_agents=num_of_robots)
            new_gnn_features = new_comm_gnn.begin()
            gnn_features_cat = torch.cat([new_gnn_features[key][subkey] for key in new_gnn_features for subkey in new_gnn_features[key]])
            reshaped_gnn_features = []
            temp = []
            c = 1
            for i in range(len(gnn_features_cat)):
                temp.append(gnn_features_cat[i].tolist())
                if c % num_of_robots == 0:
                    reshaped_gnn_features.append(torch.Tensor(temp))
                    temp = []
                c+=1
            # print(reshaped_gnn_features[0][0])
            # print("start check")
            for robot in range(num_of_robots):
                if robot not in robots_reached:
                    # prediction = model.model(torch.tensor(reshaped_gnn_features[0][robot],dtype=torch.float32).unsqueeze(0).to(device))
                    prediction = model.model(torch.tensor(reshaped_gnn_features[0][robot],dtype=torch.float32).unsqueeze(0).to(device))
                    # print(prediction)
                    predicted_class = torch.argmax(prediction, dim=1)
                    right_state = False
                    if robot == 0 and robot not in robots_reached:
                        # print(current_nodes[robot])
                        # print(predicted_class)
                        if (current_nodes[robot][0]+1 == goals[robot][0]) and (current_nodes[robot][1] == goals[robot][1]):
                            predicted_class = 3
                            right_state = True
                        path_robot_zero.append(predicted_class)
                        next_r = [current_nodes[robot][0] + actions[predicted_class][0],current_nodes[robot][1] + actions[predicted_class][1]]
                        # print(next_r)
                    next = [current_nodes[robot][0] + actions[predicted_class][0],current_nodes[robot][1] + actions[predicted_class][1]]
                    if next[0]>=20 or next[0]<0 or next[1]>=20 or next[1]<0:
                        # current_nodes[robot] = [current_nodes[robot][0] + actions[0][0],current_nodes[robot][1] + actions[0][1]]
                        pass
                    else:
                        current_nodes[robot] = [current_nodes[robot][0] + actions[predicted_class][0],current_nodes[robot][1] + actions[predicted_class][1]]
                    if (current_nodes[robot][0] == goals[robot][0]) and (current_nodes[robot][1] == goals[robot][1]):
                        robots_reached.add(robot)
                        print("trueeeeee")
            step_number = 1
            num_of_steps+=1
        if len(robots_reached) == num_of_robots:
            success += 1
            

(17, 9)
[10, 16]


C:\Users\moham\AppData\Local\Temp\ipykernel_6560\131704871.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prediction = model.model(torch.tensor(reshaped_gnn_features[0][robot],dtype=torch.float32).unsqueeze(0).to(device))


trueeeeee
(3, 6)
[1, 5]
trueeeeee
(13, 10)
[11, 16]
trueeeeee
(15, 5)
[17, 17]
trueeeeee
(9, 6)
[6, 3]
trueeeeee
(3, 6)
[6, 10]
trueeeeee
(10, 13)
[15, 14]
trueeeeee
(4, 17)
[18, 6]
trueeeeee
(18, 9)
[19, 5]
trueeeeee
(1, 7)
[12, 15]
trueeeeee
(18, 5)
[12, 11]
trueeeeee
(2, 16)
[10, 3]
trueeeeee
(4, 18)
[5, 16]
trueeeeee
(15, 5)
[10, 14]
trueeeeee
(16, 16)
[16, 3]
trueeeeee
(1, 16)
[8, 15]
trueeeeee
(18, 16)
[13, 18]
trueeeeee
(13, 17)
[11, 12]
trueeeeee
(17, 19)
[2, 8]
trueeeeee
(19, 1)
[16, 16]
trueeeeee
(12, 11)
[17, 18]
trueeeeee
(8, 5)
[8, 17]
trueeeeee
(18, 0)
[16, 17]
trueeeeee
(15, 10)
[8, 6]
trueeeeee
(19, 0)
[5, 16]
trueeeeee
(1, 16)
[6, 12]
trueeeeee
(8, 15)
[12, 8]
trueeeeee
(13, 12)
[17, 10]
trueeeeee
(6, 8)
[11, 0]
trueeeeee
(11, 10)
[10, 13]
trueeeeee
(9, 8)
[5, 0]
trueeeeee
(15, 17)
[12, 9]
trueeeeee
(11, 10)
[19, 14]
trueeeeee
(14, 9)
[9, 15]
trueeeeee
(11, 6)
[2, 13]
trueeeeee
(5, 4)
[0, 4]
trueeeeee
(7, 2)
[1, 10]
trueeeeee
(8, 12)
[16, 6]
trueeeeee
(15, 9)
[8, 18]
t

In [149]:
print(success)

100
